In [ ]:
import os
import numpy as np
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, balanced_accuracy_score
import seaborn as sns

In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader,Dataset

class BrainTumorDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths, self.labels = self.process_images(root_dir)
        self.classes = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.classes.index(self.labels[idx])
        
        if self.transform:
            image = self.transform(image)
        return image, label

    def process_images(self, path):
        images = []
        labels = []
        for category in os.listdir(path):
            category_path = os.path.join(path, category)
            if os.path.isdir(category_path):
                for img_name in os.listdir(category_path):
                    img_path = os.path.join(category_path, img_name)
                    if self.is_image_file(img_path):
                        images.append(img_path)
                        labels.append(category)
        return images, labels

    def is_image_file(self, filename):
        valid_image_extensions = [".jpg", ".jpeg", ".png", ".bmp", ".tiff"]
        return any(filename.lower().endswith(ext) for ext in valid_image_extensions)

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
}

In [ ]:
train_dataset = BrainTumorDataset('/kaggle/input/brain-tumor-data-18k/tumordata/Training', transform=data_transforms['train'])
val_dataset = BrainTumorDataset('/kaggle/input/brain-tumor-data-18k/tumordata/Testing', transform=data_transforms['val'])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

In [ ]:
import torch
from torch import nn, optim
import torchvision.models as models

class MoCo(nn.Module):
    def __init__(self, base_encoder, projection_dim=128):
        super(MoCo, self).__init__()
        self.base_encoder = base_encoder
        
        self.projection_head = nn.Sequential(
            nn.Linear(2048, 256),
            nn.ReLU(),
            nn.Linear(256, projection_dim)
        )
    
    def forward(self, x):
        features = self.base_encoder(x)
        
        features = torch.flatten(features, start_dim=1)
        
        projections = self.projection_head(features)
        
        return projections

resnet_encoder = models.resnet50(pretrained=True)
resnet_encoder = nn.Sequential(*list(resnet_encoder.children())[:-1])

moco_model = MoCo(resnet_encoder)
optimizer = optim.Adam(moco_model.parameters(), lr=1e-3)

In [ ]:
def train_and_evaluate_moco(model, train_loader, val_loader, optimizer, device, num_epochs=10):
    criterion = nn.CrossEntropyLoss()

    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    train_bacc, val_bacc = [], []
    auc_scores = []

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_train_loss = 0.0
        correct_train = 0
        total_train = 0
        y_true_train = []
        y_pred_train = []

        for x, labels in train_loader:
            x, labels = x.to(device), labels.to(device)
            optimizer.zero_grad()
            projections = model(x)
            
            loss = criterion(projections, labels)
            loss.backward()
            optimizer.step()
            
            # Accumulate metrics
            total_train_loss += loss.item()
            _, predicted = torch.max(projections, 1)
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)

            y_true_train.extend(labels.cpu().numpy())
            y_pred_train.extend(predicted.cpu().numpy())

        train_loss = total_train_loss / len(train_loader)
        train_accuracy = correct_train / total_train
        train_bacc_score = balanced_accuracy_score(y_true_train, y_pred_train)

        # Validation phase
        model.eval()
        total_val_loss = 0.0
        correct_val = 0
        total_val = 0
        y_true_val = []
        y_pred_val = []
        val_projections = []

        with torch.no_grad():
            for x, labels in val_loader:
                x, labels = x.to(device), labels.to(device)
                projections = model(x)
                
                loss = criterion(projections, labels)
                total_val_loss += loss.item()
                
                _, predicted = torch.max(projections, 1)
                correct_val += (predicted == labels).sum().item()
                total_val += labels.size(0)

                y_true_val.extend(labels.cpu().numpy())
                y_pred_val.extend(predicted.cpu().numpy())
                val_projections.append(projections.cpu().numpy())

        val_loss = total_val_loss / len(val_loader)
        val_accuracy = correct_val / total_val
        val_bacc_score = balanced_accuracy_score(y_true_val, y_pred_val)

        if len(np.unique(y_true_val)) == 2:
            val_projections = np.vstack(val_projections)
            auc_score = roc_auc_score(y_true_val, val_projections[:, 1])
            auc_scores.append(auc_score)
        else:
            auc_scores.append(np.nan)

        train_precision = precision_score(y_true_train, y_pred_train, average='macro')
        train_recall = recall_score(y_true_train, y_pred_train, average='macro')
        train_f1 = f1_score(y_true_train, y_pred_train, average='macro')

        val_precision = precision_score(y_true_val, y_pred_val, average='macro')
        val_recall = recall_score(y_true_val, y_pred_val, average='macro')
        val_f1 = f1_score(y_true_val, y_pred_val, average='macro')

        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)
        train_bacc.append(train_bacc_score)
        val_bacc.append(val_bacc_score)

        print(f'Epoch {epoch + 1}/{num_epochs}')
        print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, BAcc: {train_bacc_score:.4f}')
        print(f'Train Precision: {train_precision:.4f}, Train Recall: {train_recall:.4f}, Train F1: {train_f1:.4f}')
        print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}, BAcc: {val_bacc_score:.4f}')
        print(f'Validation Precision: {val_precision:.4f}, Validation Recall: {val_recall:.4f}, Validation F1: {val_f1:.4f}')
        if len(np.unique(y_true_val)) == 2:  # Print AUC only for binary classification
            print(f'Validation AUC: {auc_scores[-1]:.4f}')

        if (epoch + 1) % 5 == 0 or epoch == num_epochs - 1:
            plot_loss_and_accuracy(train_losses, val_losses, train_accuracies, val_accuracies, 
                                   train_bacc, val_bacc, epoch + 1)
            plot_confusion_matrix(y_true_val, y_pred_val, epoch + 1)

def plot_loss_and_accuracy(train_losses, val_losses, train_accuracies, val_accuracies, 
                           train_bacc, val_bacc, epoch, save_dir='plots'):
    epochs = range(1, epoch + 1)

    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    plt.figure(figsize=(10, 5))

    # Loss plot
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label='Train Loss')
    plt.plot(epochs, val_losses, label='Validation Loss')
    plt.title('Loss Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Accuracy plot
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label='Train Accuracy')
    plt.plot(epochs, val_accuracies, label='Validation Accuracy')
    plt.title('Accuracy Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()

    save_path = os.path.join(save_dir, f'epoch_{epoch}.png')
    plt.savefig(save_path)

    plt.show()

# Helper function to plot confusion matrix
def plot_confusion_matrix(y_true, y_pred, epoch, save_dir='plots'):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, 
                xticklabels=np.unique(y_true), yticklabels=np.unique(y_true))
    plt.title(f'Confusion Matrix at Epoch {epoch}')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')

    save_path = os.path.join(save_dir, f'confusion_matrix_epoch_{epoch}.png')
    plt.savefig(save_path)

    plt.show()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
moco_model.to(device)

In [ ]:
train_and_evaluate_moco(moco_model, train_loader, val_loader, optimizer, device, num_epochs=30)